In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("dengue-2024.csv", sep=";", encoding="latin-1")

for column in df.columns[1:]:
    df[column] = df[column].replace('-', '0')
    df[column] = pd.to_numeric(df[column], errors='coerce')

In [3]:
df = df.rename(columns={'Região de Saúde (CIR) de resid': 'Reg Saúde (CIR)/Municíp Res'})

In [4]:
df

,Reg Saúde (CIR)/Municíp Res,Em Branco,Semana 01,Semana 02,Semana 03,Semana 04,Semana 05,Semana 06,Semana 07,Semana 08,...,Semana 35,Semana 36,Semana 37,Semana 38,Semana 39,Semana 40,Semana 41,Semana 42,Semana 43,Total
0,31001 Alfenas/Machado,0,153,231,375,479,653,790,890,1578,...,22,15,20,25,17,15,14,3,0,38066
1,..... AREADO,0,80,115,220,277,345,317,298,368,...,0,0,0,0,3,2,4,0,0,3183
2,..... CONCEICAO DA APARECIDA,0,7,4,2,6,21,25,37,87,...,0,0,0,0,0,0,0,0,0,1314
3,..... DIVISA NOVA,0,2,0,1,2,13,8,27,36,...,0,0,0,0,0,0,0,0,0,460
4,..... BANDEIRA DO SUL,0,0,0,0,0,0,3,4,11,...,1,1,1,1,0,0,0,0,0,161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,..... SAO PEDRO DO SUACUI,0,0,0,1,0,3,4,11,15,...,0,0,0,0,0,0,0,0,0,247
940,..... JOSE RAYDAN,0,0,0,0,8,24,21,21,45,...,0,0,0,0,0,0,0,0,0,387
941,..... SAO JOSE DO JACURI,0,0,1,0,4,7,17,18,39,...,0,0,0,0,0,0,0,0,0,421
942,Município ignorado - MG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
pattern = r'^\d+\s[A-Za-z]+(\/[A-Za-z]+)*'
df['Reg Saúde (CIR)/Municíp Res'] = df['Reg Saúde (CIR)/Municíp Res'].replace('/',',', regex=True)

for index, row in df.iterrows():
    if re.match(pattern, row['Reg Saúde (CIR)/Municíp Res']):
        last_number = row['Reg Saúde (CIR)/Municíp Res'].split()[0].strip(' ')        
        df.at[index, 'Reg Saúde (CIR)/Municíp Res'] = \
        row['Reg Saúde (CIR)/Municíp Res'].replace(last_number+' ', last_number+'M/')
    else:
        df.at[index, 'Reg Saúde (CIR)/Municíp Res'] = \
            row['Reg Saúde (CIR)/Municíp Res'].replace('.....  ', last_number+'/')

pattern = r'^\d+M\/[A-Za-z]+(\/[A-Za-z]+)*'
municipios = df[df['Reg Saúde (CIR)/Municíp Res'].str.match(pattern)].copy()
municipios['Reg Saúde (CIR)/Municíp Res'] = municipios['Reg Saúde (CIR)/Municíp Res'].replace('M/', '/',regex=True)
residencias = df[~df['Reg Saúde (CIR)/Municíp Res'].str.match(pattern)]

In [6]:
municipios['CIR'] = municipios['Reg Saúde (CIR)/Municíp Res'].apply(lambda x: x.split('/')[0])
municipios['Município'] = municipios['Reg Saúde (CIR)/Municíp Res'].apply(lambda x: x.split('/')[-1])
municipios = municipios.drop('Reg Saúde (CIR)/Municíp Res', axis=1)
municipios = municipios.set_index('Município')

In [7]:
residencias['CIR'] = residencias['Reg Saúde (CIR)/Municíp Res'].apply(lambda x: x.split('/')[0])
residencias['Residência'] = residencias['Reg Saúde (CIR)/Municíp Res'].apply(lambda x: x.split('/')[-1])
residencias = residencias.drop('Reg Saúde (CIR)/Municíp Res', axis=1)
residencias = residencias.set_index('Residência')

/var/folders/61/gywmh3ns1m7chzq6yjjdtg540000gn/T/ipykernel_1521/1499853656.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  residencias['CIR'] = residencias['Reg Saúde (CIR)/Municíp Res'].apply(lambda x: x.split('/')[0])
/var/folders/61/gywmh3ns1m7chzq6yjjdtg540000gn/T/ipykernel_1521/1499853656.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  residencias['Residência'] = residencias['Reg Saúde (CIR)/Municíp Res'].apply(lambda x: x.split('/')[-1])


In [8]:
municipios

,Em Branco,Semana 01,Semana 02,Semana 03,Semana 04,Semana 05,Semana 06,Semana 07,Semana 08,Semana 09,...,Semana 36,Semana 37,Semana 38,Semana 39,Semana 40,Semana 41,Semana 42,Semana 43,Total,CIR
Município,,,,,,,,,,,,,,,,,,,,,
"Alfenas,Machado",0,153,231,375,479,653,790,890,1578,1711,...,15,20,25,17,15,14,3,0,38066,31001
Guaxupé,0,190,230,254,339,455,608,593,1004,1130,...,5,8,3,9,4,6,1,0,15818,31002
Itajubá,0,44,81,91,139,211,292,319,544,597,...,12,12,14,18,14,12,6,0,15648,31003
Lavras,0,40,89,95,162,302,352,356,696,800,...,17,7,8,10,8,7,5,1,21401,31004
Poços de Caldas,0,5,33,52,35,88,191,210,317,386,...,2,3,3,1,1,3,1,0,9635,31006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Salinas,0,22,64,90,167,210,386,350,455,386,...,0,0,0,0,0,0,0,0,5074,31098
"Teófilo Otoni,Malacacheta",0,66,69,111,182,279,469,501,859,1118,...,21,13,14,20,5,3,5,1,16303,31099
São Francisco,0,41,66,87,136,268,502,330,531,601,...,0,2,0,0,1,0,1,0,6581,31100


In [16]:
#municipios.to_csv('zika-2024-municipios.csv', encoding='latin-1')
# residencias.to_csv('dengue-2024-residencias.csv', encoding='latin-1')